In [1]:
%run common.ipynb


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Parameters
# Chose the variables to crate the model
input_cols = ['shape', 'profile_entrance', 'profile_exit', 'rising_entrance', 'rising_exit', 'shards', 'feathering', 'entrance_mounding', 'center_mounding', 'exit_mounding', 'mounding']
result_col = 'result'
train_dataset_file = 'DATA/Train/serration_prediction_train.xlsx'
test_dataset_file = 'DATA/Validation/serration_prediction_validation.xlsx'

In [3]:
# The data are displayed on a table
display_input_data(
    train_dataset_file,
    input_cols
)


,shape_E,shape_indeterminate,profile_entrance_V,profile_exit_V,rising_entrance_bilateral,rising_entrance_single,rising_exit_bilateral,rising_exit_single,shards_present,feathering_present,entrance_mounding_bilateral,entrance_mounding_single,center_mounding_bilateral,center_mounding_single,exit_mounding_bilateral,exit_mounding_single,mounding_marked,mounding_not marked
0,False,True,True,True,False,True,False,True,False,False,False,True,False,False,False,True,False,True
1,False,True,False,True,False,True,True,False,False,False,False,True,False,False,False,True,False,True
2,False,True,False,True,False,True,True,False,False,False,False,True,False,False,True,False,False,True
3,False,True,False,True,False,True,True,False,False,False,False,True,False,False,True,False,False,True
4,False,True,False,True,False,True,True,False,False,False,False,True,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,True,False,True,True,False,True,False,True,False,False,False,True,False,False,False,True,True,False
276,True,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False
277,True,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
278,True,False,True,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [4]:
# The two datasets are tested for compatibility 
check_input_data_indices(train_dataset_file, test_dataset_file, input_cols)

Datasets are compatible


In [5]:
# Logistic regression model
model = train_model(
    train_dataset_file,
    input_cols,
    result_col
)
# model
print('b0 = ' + str(model.intercept_))
print('b1...n = ' + str(model.coef_))

b0 = [2.94177671]
b1...n = [[-1.21639648 -1.28412923 -0.23924146 -2.3325252  -1.73212203 -1.63566348
  -1.3639484  -0.16607839  1.6963423   1.68430953 -0.56727603 -0.18639544
  -0.30974718 -0.53760101 -0.37406523  1.41657166  0.46015146  0.21973993]]


In [6]:
# Test for overfitting
# Evaluation of the training
x_train, y_train = prepare_dataset(train_dataset_file, input_cols, result_col)
model.score(x_train, y_train)

0.9

In [7]:
# 11 variables were used 
evaluate_model(
    model,
    test_dataset_file,
    input_cols,
    result_col
)

0.7164179104477612

In [9]:
# filter data for the new dataframe
df = pd.read_excel("DATA/Train/serration_prediction_train.xlsx")
x_df = df[input_cols].apply(lambda x: x.str.strip())
x = pd.get_dummies(x_df, drop_first=True)

result = model.predict(x)
print(result)
# this paragraph was created to see if serrated or non serrated corresponded either False or True
# True corresponds to serrated

[False False False False False False False False  True False False False
 False  True False False False False False False False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False  True  True  True  True  T

In [15]:
# filter data for the new dataframe
df = pd.read_excel("DATA/Validation/serration_prediction_validation.xlsx")
label = df["label"]
string_expresult = df["result"]

x_df = df[input_cols].apply(lambda x: x.str.strip())
x = pd.get_dummies(x_df, drop_first=True)

result = model.predict(x)

print(string_expresult)
print(result)


0     non-serrated
1     non-serrated
2     non-serrated
3     non-serrated
4     non-serrated
          ...     
62    non-serrated
63    non-serrated
64    non-serrated
65    non-serrated
66    non-serrated
Name: result, Length: 67, dtype: object
[ True  True  True False False False False False False False  True  True
  True False  True  True False False False  True  True  True  True  True
 False False False  True  True False False False False False False False
 False False False  True  True  True  True  True  True  True  True  True
  True  True  True False False False  True  True  True False False False
 False False  True  True  True  True False]


In [20]:
# to organize better the things, we have to create a new list for the "result" and transform them in boolean

expected_result = []
for x in string_expresult:

    if x == "serrated":
        expected_result.append(True)
    
    else:
        expected_result.append(False)

print(expected_result)



[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False]


In [35]:
#build dataframe with result, expected result and label 

data = {'label': label, 'result': result, "expected_result" : expected_result}  
  
# Create DataFrame  
df = pd.DataFrame(data)  
   
print(df)  

grouped_df = df.groupby(by = "label")

print(grouped_df.head())


    

    label  result  expected_result
0       2    True            False
1       2    True            False
2       2    True            False
3       2   False            False
4       2   False            False
..    ...     ...              ...
62      1    True            False
63      1    True            False
64      1    True            False
65      1    True            False
66      1   False            False

[67 rows x 3 columns]
    label  result  expected_result
0       2    True            False
1       2    True            False
2       2    True            False
3       2   False            False
4       2   False            False
10      4    True            False
11      4    True            False
12      4    True            False
13      4   False            False
14      4    True            False
20      7    True             True
21      7    True             True
22      7    True             True
23      7    True             True
24      7   False             Tr

In [34]:
for name, sub_df in grouped_df:
    print(f'{name}\n{sub_df}\n')

1
    label  result  expected_result
57      1   False            False
58      1   False            False
59      1   False            False
60      1   False            False
61      1   False            False
62      1    True            False
63      1    True            False
64      1    True            False
65      1    True            False
66      1   False            False

2
   label  result  expected_result
0      2    True            False
1      2    True            False
2      2    True            False
3      2   False            False
4      2   False            False
5      2   False            False
6      2   False            False
7      2   False            False
8      2   False            False
9      2   False            False

3
    label  result  expected_result
29      3   False            False
30      3   False            False
31      3   False            False
32      3   False            False
33      3   False            False
34      3   False      

In [63]:
for name, sub_df in grouped_df:
    tot_corr = 0
    for index, row in sub_df.iterrows():
        if row["result"] == row["expected_result"]:
            tot_corr += 1
    tot = len(sub_df)
    print( "knife:", name) 
    #print("total correct:", tot_corr)
    #print('total:', tot) 
    
    accuracy_perc = (tot_corr / tot ) * 100
    
    print( "accuracy %:", accuracy_perc, "\n")
       


knife: 1
accuracy %: 60.0 

knife: 2
accuracy %: 70.0 

knife: 3
accuracy %: 100.0 

knife: 4
accuracy %: 40.0 

knife: 5
accuracy %: 100.0 

knife: 6
accuracy %: 66.66666666666666 

knife: 7
accuracy %: 66.66666666666666 



In [62]:
# Model optimisation, search for the optimal parameter combination
# Printing score best and worst model
run_input_optimizer(train_dataset_file, test_dataset_file, input_cols, result_col)

Testing combinations of length 2
Testing combinations of length 3
Testing combinations of length 4
Testing combinations of length 5
Testing combinations of length 6
Testing combinations of length 7
Testing combinations of length 8
Testing combinations of length 9
Testing combinations of length 10
Testing combinations of length 11
Max score: 0.8805970149253731
Best columns combination: ('shape', 'profile_entrance', 'rising_entrance', 'shards', 'mounding')
Min score: 0.4626865671641791
Worse columns combination: ('feathering', 'entrance_mounding')


In [21]:
import ipywidgets as widgets


# tex = "$$p = {\frac{1}{1 + e ^ {-(\beta_0 + \beta_1X_i + \beta_2X_2 + \beta_3X_3 + \beta_4X_4 + \beta_5X_5)}}}$$"
# tex = "$$p = {\frac{1}{1 + e ^ {-(3 + 2X_1 + \beta_2X_2 + \beta_3X_3 + \beta_4X_4 + \beta_5X_5)}}}$$"
tex = r"$$p = {\frac{1}{1 + e ^ {-("

# Add intercept
tex += str(np.round(model.intercept_[0], decimals=2))

 
# Add coefficients
for i, b in enumerate(model.coef_[0]):
    if b < 0:
        sign = '-'
    elif b > 0:
        sign = '+'
    else:
        continue
    tex += f"{sign}{str(np.abs(np.round(b, decimals=2)))}x_{{{str(i+1)}}}"

tex += ")}}}$$"

widgets.HTMLMath(
    value=tex,
    placeholder='Logistic regression',
)

HTMLMath(value='$$p = {\\frac{1}{1 + e ^ {-(2.94-1.22x_{1}-1.28x_{2}-0.24x_{3}-2.33x_{4}-1.73x_{5}-1.64x_{6}-1…